In [ ]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
from crawlers.models import *
from crawlers.finlab.pioneers import *
from crawlers.finlab.import_tools import *
import time
pd.set_option("display.max_rows", None)

In [ ]:
class CrawlBrokerTradeTW:
    
    def __init__(self,start_date):
        self.start_date = start_date
        self.start_date_str = start_date.strftime("%Y-%m-%d")
        self.target_name = "台股分點進出資訊"
        self.format = "time_series"
        
    
    def check_trade_day(self):
        stock_range=CrawlStockPriceTW(self.start_date)
        try:
            df=list(pd.concat([stock_range.crawl_sii(),stock_range.crawl_otc()])['stock_id'].values)
            return df
        except ValueError:
            return None
        
    def broker_trade(self,stock_id):
        print(stock_id)
        url='http://jsjustweb.jihsun.com.tw/z/zc/zco/zco.djhtm?a='+stock_id+'&e='+self.start_date_str+'&f='+self.start_date_str
        # http://jsjustweb.jihsun.com.tw/z/zc/zco/zco.djhtm
        # https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco.djhtm
        r = requests.post(url)
        html_df = pd.read_html(StringIO(r.text))
        df = pd.DataFrame(html_df[2])
        #holiday
        if len(df)<9:
            return  None
        df.columns=df.iloc[5]
        buy_net_avg_cost=float(df[df['買超券商']=='平均買超成本']['買超'].values[0])
        sell_net_avg_cost=float(df[df['賣超券商']=='平均賣超成本']['賣超'].values[0])
        df=df.iloc[6:-3]
        buy_side=df.iloc[:,:5]
        buy_side=buy_side.rename(columns={'買超券商':'broker_name','買進':'buy_num',
                                          '賣出':'sell_num','買超':'net_bs',
                                          '佔成交比重':'transactions_pt'})

        sell_side=df.iloc[:,5:]
        sell_side=sell_side.rename(columns={'賣超券商':'broker_name','買進':'buy_num',
                                            '賣出':'sell_num','賣超':'net_bs',
                                            '佔成交比重':'transactions_pt'})

        df_all=pd.concat([buy_side,sell_side],sort=False).dropna()
        df_all.iloc[:,1:]=df_all.iloc[:,1:].apply(lambda s: pd.to_numeric(s.str.replace('%',''), errors="coerce"))
        df_all['net_bs']=df_all['buy_num']-df_all['sell_num']
        df_all['net_bs_cost']=[i*buy_net_avg_cost if i>0 else i*sell_net_avg_cost for i in df_all['net_bs']]
        df_all['date'] = pd.to_datetime(self.start_date).date()
        df_all['broker_name']=df_all['broker_name'].apply(lambda s:s.replace('證券','')).apply(lambda s:s.replace('(牛牛牛)','犇'))
        df_all['stock_id']=str(stock_id)
        AddToSQL.add_to_sql(BrokerTradeTW, df_all,['stock_id','date','broker_name'], [{'broker_name':'broker_name'},{'stock_id':'stock_id'}])
        
        return  df_all

    def crawl_main(self):
        crawl_list=self.check_trade_day()
        if crawl_list is not None:
            new_obj=BrokerTradeTW.objects.filter(date= self.start_date_str)
            if len(new_obj)>0:
                table_last_day=table_latest_date(engine, BrokerTradeTW._meta.db_table)
                finish_obj=BrokerTradeTW.objects.filter(date=table_last_day)
                last_stock_id=finish_obj[len(finish_obj)-1].stock_id
                crawl_list=crawl_list[crawl_list.index(last_stock_id):]
            for stock_id in crawl_list:
                 self.broker_trade(stock_id)
        else:
            pass



# a=CrawlBrokerTradeTW(datetime.datetime(2020,4,23))
# a.broker_trade('00774B')
# df=broker_trade(stock_id,datetime.datetime(2020,4,23),end_time=None)
# df

a=CrawlBrokerTradeTW(datetime.datetime(2018,1,29))
# df=a.broker_trade('6263')
# df
a.crawl_main()

In [ ]:
table_last_day=table_latest_date(engine, BrokerTradeTW._meta.db_table)
finish_obj=BrokerTradeTW.objects.filter(date=table_last_day)
last_stock_id=finish_obj[len(finish_obj)-1].stock_id
last_stock_id

In [ ]:
table_last_day=table_latest_date(engine, BrokerTradeTW._meta.db_table)
table_last_day

In [ ]:
datetime.datetime.strptime('2018-01-03', '%Y-%m-%d')